In [2]:
import IPython
import pandas as pd
import numpy as np

from datetime import datetime, date
from math import *
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6
import re
from decimal import Decimal

from os import listdir
from os.path import isfile, join

In [ ]:
def load_candles(filename):
  df = pd.read_json(filename)
  df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
  # df.columns = ['timestamp', 'open']
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=False)
  # df = df[(df['date']>=date(2020,4,6)) & (df['date']<=date(2021,4,5))] 
  # df['timestamp'] = df.timestamp.astype('datetime64[m]')
  df.set_index(pd.DatetimeIndex(df["timestamp"]), inplace=True, drop=True)
  # df = df['2020-04-06' : '2021-04-05']
  df.drop(['timestamp', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)
  # df['ohl3'] = (df['open'] + df['high'] + df['low']) / 3
  return df

# datapath = './data'
# kline_pairs = [f.split('.')[0].split('-')[0] for f in listdir(datapath) if isfile(join(datapath, f))]
# klines = {pair: load_candles(f'./data/{pair}-1m.json') for pair in kline_pairs}
# klines


In [ ]:
p = qpairs[0]
filename = f'./freq-user-data/data/binance/{p}-1m.json'
df = pd.read_json(filename)
df.columns = ['ts', 'open', 'high', 'low', 'close', 'volume']
df

In [ ]:

datapath = './freq-user-data/data/binance/'
kline_pairs = [f.split('.')[0].split('-')[0] for f in listdir(datapath) if isfile(join(datapath, f))]
# klines = {pair: load_candles(f'./data/{pair}-1m.json') for pair in kline_pairs}
kline_pairs


In [ ]:
qpairs = [
 'BTC_USDT',
 'ETH_USDT',
 'GBP_USDT']
timeframes = ['1m']

In [ ]:


for p in qpairs:
    for tf in timeframes:
        print(f'Converting {p} - {tf}...')
        pd.read_json(f'./freq-user-data/data/binance/{p}-{tf}.json'
            ).set_axis(['dt','o','h','l','c','v'], axis=1
            ).set_index('dt'
            ).to_csv(f'./data-csv/{p}-{tf}.csv')


In [ ]:
p = qpairs[0]
filename = f'./freq-user-data/data/binance/{qpairs[0]}-1m.json'
df = pd.read_json(filename
).set_axis(['dt','o','h','l','c','v'], axis=1
).set_index('dt')
# df.columns = [
df

In [ ]:
curl -H "X-CMC_PRO_API_KEY: 733a53cb-19a4-4070-8291-a03aa07da1ad" -H "Accept: application/json" -d "" -G "https://pro.coinmarketcap.com/v1/cryptocurrency/listings/latest"


In [ ]:

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  # 'start':'1',
  # 'limit':'5000',
  # 'convert':'USD'
  'symbol': 'BTC,ETH,ADA'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '733a53cb-19a4-4070-8291-a03aa07da1ad',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

## CMC Historical data

In [4]:
import configparser
import json
import pytz
from datetime import datetime
from requests import Request, Session
from dateutil import parser

symbols = ['BTC,GBP,ETH,XRP,ALGO,ADA,LINK,BNB,DOT,XLM,LTC,AVAX,AAVE,EGLD,ATOM,DOGE,GRT,ENJ,CAKE,TRX,BUSD,LUNA,USDC,USDT,SOL,MATIC,FTM,FIL,ICP,CRV,REN,RUNE,KSM,ZIL,THETA,LRC,MANA,SHIB,ILV,GALA,AXS,SAND,RNDR,CHZ,APT,ARB,OP,RAY,INJ,BEAM,NEAR,ALU,KAS,TIA,PYR,SNX,MYRIA,IMX,COQ,QNT,TET,AMP']
convert = ['USDT' ]

# def get_info():
    # Read the API key from the coinmarket.ini file
    # config = configparser.ConfigParser()
    # config.read('coinmarket.ini')
    # api_key = config['DEFAULT']['API_KEY']

    # Set up the request parameters and headers
# url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/ohlcv/historical'

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
parameters = { 
    # 'slug': 'bitcoin', 
    'symbol': symbols,
    'convert': convert,
    # 'time_start': '2020-01-01'
                }
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': '733a53cb-19a4-4070-8291-a03aa07da1ad'
}

# Send the request and retrieve the response
session = Session()
session.headers.update(headers)
response = session.get(url, params=parameters)
data = json.loads(response.text)

pd.DataFrame.from_records([{'asset':c, **data['data'][c]['quote']['USDT']} for  c in data['data'].keys()]).to_csv('cmc-quotes.csv')

In [83]:
pd.DataFrame.from_records([{'asset':c, **data['data'][c]['quote']['USDT']} for  c in data['data'].keys()]).to_csv('cmc-quotes.csv')
    

## Convert JSON candlesticks to csv

In [ ]:
from glob import glob
import pandas as pd
import os
    
freqdatadir = '/media/grenada/Data/Accounting/pk-crypto-tax-calculator/freq-user-data/data'
csvdatadir = '/media/grenada/Data/Accounting/pk-crypto-tax-calculator/data-csv'
xcgs = [x.split('/')[-1] for x in glob(f"{freqdatadir}/*", recursive = False)]
for xcg in xcgs:
    djs = f'{freqdatadir}/{xcg}'
    dcsv = f'{csvdatadir}/{xcg}'
    if not os.path.exists(dcsv):
        os.makedirs(dcsv)
    print('Processing folder: ', djs)
    
    for js in [x.split('/')[-1].split('.')[0] for x in glob(f"{djs}/*")]:
        # fcsv = '{csvdatadir}/`'
        print(f'Converting {xcg} - {js} ...')
        fcsv = f'{dcsv}/{js}.csv'
        if not os.path.exists(fcsv):
            pd.read_json(f'{freqdatadir}/{xcg}/{js}.json'
                ).set_axis(['dt','o','h','l','c','v'], axis=1
                ).set_index('dt'
                ).to_csv(fcsv)
        # print(js.split('.')[0])
    # print(jsfiles)


In [9]:
import statistics as sta

sta.variance([i+1 for i in range(9)])

7.5